In [1]:
import tensorflow as tf
import utils.deeplabv3_ as build_model
import cv2
import numpy as np
import glob
import csv
import datetime
import utils.draw_predict as dp
import utils.call_data as cd

# video_path_root = 'F:/07_Data/eye2/total/'
# avi_write_path_root = 'Z:/segmentation_pupil_v3/avi/'
model_path_root = './model/113'
# csv_path_root = 'Z:/segmentation_pupil_v3/csv/'

class_num = 2
c_area_thr = 400
ratio_thr = 0.4
diff_thr = 350

roi_w = 170
roi_h = 80
refer_frame = 10

font = cv2.FONT_HERSHEY_SIMPLEX
fontScale = 0.5

In [2]:
# tensorflow
tf.reset_default_graph()
X = tf.placeholder(tf.float32, shape=[None, None, None, 3])
model = build_model.deeplabv3(X, class_num, True)

sess = tf.Session()
sess.run(tf.global_variables_initializer())
saver = tf.train.Saver()
saver.restore(sess, tf.train.latest_checkpoint(model_path_root))

W0219 10:59:19.119752 172428 deprecation_wrapper.py:119] From C:\Users\th_k9\Desktop\code\utils\deeplabv3_.py:154: The name tf.variable_scope is deprecated. Please use tf.compat.v1.variable_scope instead.

W0219 10:59:19.124738 172428 deprecation_wrapper.py:119] From C:\Users\th_k9\Desktop\code\utils\deeplabv3_.py:155: The name tf.get_variable is deprecated. Please use tf.compat.v1.get_variable instead.

W0219 10:59:19.125735 172428 deprecation.py:506] From C:\Users\th_k9\AppData\Local\Continuum\anaconda3\envs\Kimtae\lib\site-packages\tensorflow\python\ops\init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0219 10:59:19.146680 172428 deprecation_wrapper.py:119] From C:\Users\th_k9\Desktop\code\utils\deeplabv3_.py:168: The name tf.nn.max_pool is deprecated. Please use

# 후처리 알고리즘 손보기

In [18]:
# opencv
# cap = cv2.VideoCapture(1)
cap = cv2.VideoCapture('./data/7_after_b_1.avi')

f_n = 0
while(cap.isOpened()):
    ret, frame = cap.read()

    if ret:
        seg = np.ones((480, 640, 3)).astype(np.uint8)
        
#         frame = frame[60:420]
        frame = cv2.resize(frame, (640, 480))
        
        ori_frame = frame.copy()
        s_frame = frame.copy()

        in_x = np.expand_dims(frame, axis=0)
        predict = sess.run([model], feed_dict = {X: in_x})
        segmentation = np.argmax(predict[0][0], 2)
        segmentation = np.where(segmentation == 1, 255, 0)
        segmentation = segmentation.astype(np.uint8)
        y, x = np.where(segmentation[:, :] == 255)
        s_frame[y, x] = (0, 0, 255)
        seg[y, x] = (255, 255, 255)
        
        cv2.imshow('testt', seg)

        pupil_cand_list = []
        if f_n == 0:
            pre_segment = len(y)
        else:
            diff = len(y) - pre_segment

            if abs(diff) < diff_thr:
                contours, hierachy = cv2.findContours(segmentation, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

                for cnt in contours:
                    c_area = cv2.contourArea(cnt)

                    if c_area > c_area_thr:
                        _, _, w, h = cv2.boundingRect(cnt)

                        if (float(w) / h) > ratio_thr:
                            ellipse = cv2.fitEllipse(cnt)

                            M = cv2.moments(cnt)
                            cx = int(M['m10']/M['m00'])
                            cy = int(M['m01']/M['m00'])

                            pupil_cand_list.append([c_area, ellipse, (cx, cy)])

#         cv2.putText(frame, str(f_n), (30, 40), font, fontScale, (0, 0, 255), 2)
        # 최종 동공 정보 (크기, 타원, 중심)
        if len(pupil_cand_list) != 0:
            pupil = max(pupil_cand_list)
            cv2.ellipse(frame, pupil[1], (0, 0, 255), 2)
#             cv2.putText(frame, str(pupil[0]), (30, 60), font, fontScale, (0, 0, 255), 2)
            cv2.circle(frame, (cx, cy), 4, (255, 0, 0), -1)
        else:
            pupil = [0, 0, (0, 0)]

#         w_csv = []
#         w_csv.append(f_n)
#         w_csv.append(pupil[0])
#         w_csv.append(pupil[2])
#         wr.writerow(w_csv)
        
#         print('x : {}, y : {}'.format(cx, cy))
        
#         cv2.putText(s_frame, str(len(y)), (30, 80), font, fontScale, (0, 0, 255), 2)
#         s_frame = cv2.resize(s_frame, (320, 240))
#         frame = cv2.resize(frame, (320, 240))
#         write_frame = cv2.hconcat([s_frame, frame])

#         cv2.imshow('write_frame', write_frame)
        
        ori_frame = cv2.resize(ori_frame, (320, 240))
        seg = cv2.resize(seg, (320, 240))
        frame = cv2.resize(frame, (320, 240))
        write_frame2 = cv2.hconcat([ori_frame, seg, frame])
        cv2.imshow('2', write_frame2)
        cv2.imwrite('./save/' + str(f_n) + '.jpg', write_frame2)

        f_n += 1

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    else:
        break

cap.release()
cv2.destroyAllWindows()

In [26]:
import turtle as t

t.forward(50)